# Cafe Sales Analytics Projects

## Objectives
fill the missing values in the cleaned datasets Cafe Sales.

### SETUP for Filling values

In [1]:
import pandas as pd
import numpy as np

In [2]:
#import CSV
data_path = "../data/processed/cleaned_cafe_sales_without_filling.csv"
try:
    df = pd.read_csv(data_path,
                     dtype= {
                        "Transaction ID" : "object",
                        "Item" : "string",
                        "Quantity": "Int32",
                        "Price Per Unit": "float64",
                        "Total Spent": "float64",
                        "Payment Method": "string", 
                        "Location": "string", 
                     } 
                     ,parse_dates=["Transaction Date"]
                     )
except FileNotFoundError:
    print("ERROR : File Not Found")

In [3]:
#make a copy to fill
fill_df = df.copy()
pd.set_option("display.max_row", None)

In [4]:
fill_df.columns

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
      dtype='object')

## Fill missing values in 'Item' 
Missing Item names were inferred only using the price per unit column and only when an item's price per unit is unique.
items that have the same prices where left out to avoid assumptions.

From 969 missing values in the item columns to 480 missing values. I was able to infer a total of 489 item names first based on the Price per unit column followed by dividing the total spent column with the quantity column to figure out the price per unit and therefore the missing item values. 

In [5]:
print(df["Item"].isnull().sum())

969


In [6]:
#find known items
item_known = fill_df[fill_df["Item"].notna()]

In [7]:
# find items with unique price per unit
price_count = item_known.groupby("Price Per Unit")["Item"].nunique()
print(price_count)

Price Per Unit
1.0    1
1.5    1
2.0    1
3.0    2
4.0    2
5.0    1
Name: Item, dtype: int64


In [8]:
# get safe items to infer name from the price per unit == 1
safe_price = price_count[price_count == 1].index
safe_price

Index([1.0, 1.5, 2.0, 5.0], dtype='float64', name='Price Per Unit')

In [9]:
# create a lookup table
price_to_item = (item_known[item_known["Price Per Unit"].isin(safe_price)]
                 .drop_duplicates("Price Per Unit")
                 .set_index("Price Per Unit")["Item"])
print(price_to_item)

Price Per Unit
2.0    Coffee
1.0    Cookie
5.0     Salad
1.5       Tea
Name: Item, dtype: string


In [10]:
# find missing values in the item column
mask = fill_df["Item"].isna()
# fill missing values using the mask
fill_df.loc[mask,"Item"] = ( fill_df.loc[mask,"Price Per Unit"].map(price_to_item) )

In [11]:
print(fill_df["Item"].isnull().sum())

501


### Fill missing values in 'Item' Using Total Spent and Quantity

Missing Item names were inferred using the Total Spent and Quantity column to get the price per unit.

In [12]:
# create a temp df for calculated price
fill_df["Computed_price"] = (fill_df["Total Spent"]/ fill_df["Quantity"]).round(2)
fill_df.columns

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date', 'Computed_price'],
      dtype='object')

In [13]:
# find missing values in the item column
mask = (fill_df["Item"].isna() 
        &  
        fill_df["Computed_price"].isin(safe_price)
        )
# fill missing values using the mask
fill_df.loc[mask,"Item"] = ( fill_df.loc[mask,"Computed_price"].map(price_to_item) )

In [14]:
print(fill_df["Item"].isnull().sum())

480


## Fill missing values in 'Price per unit' 

Missing values in price per unit were infered using the item column and the computation of total spent/quantity columns

From 533 missing values in the price per unit column to 6 missing values. These values were infered by first using the item column and the using the computation total spent divided by quantity to fill the remaining values if both values were not null and valid

In [15]:
print(fill_df["Price Per Unit"].isna().sum())

533


In [16]:
# find all not Null values in Item and Price per unit
valid = fill_df[fill_df["Item"].notna() & fill_df["Price Per Unit"].notna()]

In [17]:
# find out if each item has a unique price 
price_count = valid.groupby("Item")["Price Per Unit"].nunique()
price_count

Item
Cake        1
Coffee      1
Cookie      1
Juice       1
Salad       1
Sandwich    1
Smoothie    1
Tea         1
Name: Price Per Unit, dtype: int64

In [18]:
# get items that sell at one price (safe)
safe_item = price_count[price_count == 1].index
safe_item

Index(['Cake', 'Coffee', 'Cookie', 'Juice', 'Salad', 'Sandwich', 'Smoothie',
       'Tea'],
      dtype='string', name='Item')

In [19]:
#create a lookup table item -> prices
item_to_price = (
    valid[valid["Item"].isin(safe_item)]
    .drop_duplicates("Item")
    .set_index("Item")["Price Per Unit"]
)
print(item_to_price)

Item
Coffee      2.0
Cake        3.0
Cookie      1.0
Salad       5.0
Smoothie    4.0
Sandwich    4.0
Tea         1.5
Juice       3.0
Name: Price Per Unit, dtype: float64


In [20]:
# fill missing values in price per unit
mask = (fill_df["Price Per Unit"].isna() 
        &  
        fill_df["Item"].isin(item_to_price.index)
        )
# fill missing values using the mask
fill_df.loc[mask,"Price Per Unit"] = ( fill_df.loc[mask,"Item"].map(item_to_price) )

In [21]:
print(f"Null values in Price Per Unit: {fill_df['Price Per Unit'].isna().sum()},\nNull values in Item : {fill_df['Item'].isna().sum()}")

Null values in Price Per Unit: 33,
Null values in Item : 480


### Fill in missing values in price per unit using total spent/quantity

Remaining missing values were infered using total spent divided by quantity if both values were not null and valid



In [22]:
#create a mask for safe rows
mask = (
    fill_df["Price Per Unit"].isna() &
    fill_df["Total Spent"].notna() &
    fill_df["Quantity"].notna() &
    (fill_df["Quantity"] > 0)
    )

In [23]:
# fill missing values in price per unit
fill_df.loc[mask,"Price Per Unit"] = ( fill_df.loc[mask,"Computed_price"].round(2) )

In [24]:
fill_df.drop(columns="Computed_price", inplace= True)
print(fill_df['Price Per Unit'].isna().sum())

6


## Fill missing values in 'Quantity' 
Missing values in Quantity were infered using the computation of total spent/price per unit columns

From 479 missing values in the quantity column to 23 NaN. This was achieved by infering the quantity using the total spent divided by the price per unit when both values were not null and price per unit is greater than zero. Also making sure the computation was rounded to be an integer. Only affected the quantity values that were null and fufilled the aformention conditions.

In [25]:
print(fill_df["Quantity"].isna().sum())

479


In [26]:
# create a temp df for calculated quantity
fill_df["Computed_quantity"] = (fill_df["Total Spent"]/ fill_df["Price Per Unit"])

In [27]:
fill_df.columns

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date', 'Computed_quantity'],
      dtype='object')

In [28]:
# create a mask for the proper conditions
mask = (
    fill_df["Quantity"].isna() &
    fill_df["Total Spent"].notna() &
    fill_df["Price Per Unit"].notna() &
    (fill_df["Price Per Unit"] > 0) &
    (fill_df["Computed_quantity"].round() == fill_df["Computed_quantity"])
)

In [29]:
#fill in missing Quantities
fill_df.loc[mask,"Quantity"] = ( fill_df.loc[mask,"Computed_quantity"]).astype(int)

In [30]:
#check changes
print(fill_df["Quantity"].isna().sum())

23


In [31]:
fill_df.drop(columns= "Computed_quantity",inplace= True)
fill_df.columns

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
      dtype='object')

In [32]:
fill_df.loc[fill_df["Quantity"].isna() ,["Item", "Quantity", "Price Per Unit", "Total Spent"]].head(23)

,Item,Quantity,Price Per Unit,Total Spent
236,Salad,<NA>,5.0,NaN
278,Juice,<NA>,3.0,NaN
641,Juice,<NA>,3.0,NaN
738,Sandwich,<NA>,4.0,NaN
2796,Cake,<NA>,3.0,NaN
3203,Smoothie,<NA>,4.0,NaN
3224,Coffee,<NA>,2.0,NaN
3401,Tea,<NA>,1.5,NaN
3779,<NA>,<NA>,NaN,25.0
4257,Coffee,<NA>,2.0,NaN


## Fill missing values in 'Total Spent' 
Missing values in Total Spent were infered using the computation of Quantity * price per unit columns

From 503 missing values in the total spent column to 23 NaN. This was achieved by infering the total spent using the quantity  multiplied by the price per unit when both values were not null and greater than zero. Also making sure the computation was rounded to 2 decimal. Only affected the total spent values that were null and fufilled the aformention conditions.

In [33]:
print(fill_df["Total Spent"].isna().sum())

502


In [34]:
# create a temp df for calculated total spent
fill_df["Computed_total"] = (fill_df["Quantity"] * fill_df["Price Per Unit"])

In [35]:
# create a mask to check all conditions
mask = (
    fill_df["Total Spent"].isna() &
    fill_df["Quantity"].notna() &
    fill_df["Price Per Unit"].notna() &
    (fill_df["Quantity"] > 0) &
    (fill_df["Price Per Unit"] > 0)
)

In [36]:
# fill in the null values in total spent 
fill_df.loc[mask, "Total Spent"] = (fill_df.loc[mask, "Computed_total"].round(2))

In [37]:
# check changes
print(fill_df["Total Spent"].isna().sum())

23


In [38]:
fill_df.drop(columns= "Computed_total", inplace = True)

In [39]:
fill_df.columns

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
      dtype='object')

## Save the infered dataset to a new CSV

In [40]:
fill_df.to_csv("../data/processed/infered_cafe_sales_completed.csv",index = False)